In [2]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=AUZWyThfl5cObDMRt97J8zMk12LPMS3Fww6OCvOb"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-04-21T10:48:51+00:00","competitions":[{"id":"sr:competition:620","name":"Hopman Cup","type":"mixed","gender":"mixed","category":{"id":"sr:category:181","name":"Hopman Cup"}},{"id":"sr:competition:660","name":"World Team Cup","type":"mixed","gender":"men","category":{"id":"sr:category:3","name":"ATP"},"level":"atp_250"},{"id":"sr:competition:990","name":"ATP Challenger Tour Finals","parent_id":"sr:competition:6239","type":"singles","gender":"men","category":{"id":"sr:category:72","name":"Challenger"}},{"id":"sr:competition:1207","name":"Championship International Series","type":"singles","gender":"women","category":{"id":"sr:category:6","name":"WTA"},"level":"wta_championships"},{"id":"sr:competition:2100","name":"Davis Cup","type":"mixed","gender":"men","category":{"id":"sr:category:76","name":"Davis Cup"}},{"id":"sr:competition:2102","name":"Billie Jean King Cup","type":"mixed","gender":"women","category":{"id":"sr:category:74","name":"Billie Jean King Cup"}},{"i

In [4]:
import json
data = json.loads(response.text)
data


{'generated_at': '2025-04-21T10:48:51+00:00',
 'competitions': [{'id': 'sr:competition:620',
   'name': 'Hopman Cup',
   'type': 'mixed',
   'gender': 'mixed',
   'category': {'id': 'sr:category:181', 'name': 'Hopman Cup'}},
  {'id': 'sr:competition:660',
   'name': 'World Team Cup',
   'type': 'mixed',
   'gender': 'men',
   'category': {'id': 'sr:category:3', 'name': 'ATP'},
   'level': 'atp_250'},
  {'id': 'sr:competition:990',
   'name': 'ATP Challenger Tour Finals',
   'parent_id': 'sr:competition:6239',
   'type': 'singles',
   'gender': 'men',
   'category': {'id': 'sr:category:72', 'name': 'Challenger'}},
  {'id': 'sr:competition:1207',
   'name': 'Championship International Series',
   'type': 'singles',
   'gender': 'women',
   'category': {'id': 'sr:category:6', 'name': 'WTA'},
   'level': 'wta_championships'},
  {'id': 'sr:competition:2100',
   'name': 'Davis Cup',
   'type': 'mixed',
   'gender': 'men',
   'category': {'id': 'sr:category:76', 'name': 'Davis Cup'}},
  {'id'

In [6]:
competitions=data['competitions']
competitions_list=[]
for comp in competitions:
    competitions_list.append({
        "competition_id":comp["id"],
        "competition_name":comp["name"],
        "parent_id":comp.get("parent_id"),
        "type":comp["type"],
        "gender":comp["gender"],
        "category_id":comp["category"]["id"]
    })
    

In [8]:
import pandas as pd
df1=pd.DataFrame(competitions_list)
df1.head()

,competition_id,competition_name,parent_id,type,gender,category_id
0,sr:competition:620,Hopman Cup,None,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,None,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,None,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,None,mixed,men,sr:category:76


In [10]:
categories_list=[]
for comp in competitions:
    categories_list.append({
        "category_id":comp["category"]["id"],
        "category_name":comp["category"]["name"]
    })
    

In [12]:
df2=pd.DataFrame(categories_list)
df2.head()

,category_id,category_name
0,sr:category:181,Hopman Cup
1,sr:category:3,ATP
2,sr:category:72,Challenger
3,sr:category:6,WTA
4,sr:category:76,Davis Cup


In [14]:
!pip install mysql-connector-python
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # port=3306
)

mycursor = mydb.cursor(buffered=True)


In [15]:
mycursor.execute("CREATE DATABASE project")

DatabaseError: 1007 (HY000): Can't create database 'project'; database exists

In [18]:
mycursor.execute("use project")

In [20]:
mycursor.execute("create table Categories(category_id varchar(50) Primary key,category_name varchar(100) Not null)")

In [22]:
insert_query = "INSERT IGNORE INTO Categories (category_id, category_name) VALUES (%s, %s)"
for index, row in df2.iterrows():
    mycursor.execute(insert_query, (row['category_id'], row['category_name']))
mydb.commit()

In [24]:
'''insert_query = """
INSERT INTO Categories (category_id, category_name)
VALUES (%s, %s)
ON DUPLICATE KEY UPDATE category_name = VALUES(category_name)
"""'''

'insert_query = """\nINSERT INTO Categories (category_id, category_name)\nVALUES (%s, %s)\nON DUPLICATE KEY UPDATE category_name = VALUES(category_name)\n"""'

In [26]:
mycursor.execute("""
CREATE TABLE Competitions (
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(50) NOT NULL,
    parent_id VARCHAR(50),
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
)
""")

In [30]:
insert_query = "INSERT IGNORE INTO Competitions (competition_id, competition_name, parent_id, type, gender, category_id) VALUES (%s, %s, %s, %s, %s, %s)"
for index, row in df1.iterrows():
    mycursor.execute(insert_query, (
        row['competition_id'],
        row['competition_name'],
        row['parent_id'],
        row['type'],
        row['gender'],
        row['category_id']
    ))
mydb.commit()

In [50]:
#1.List all competitions along with their category name

!pip install tabulate
from tabulate import tabulate
mycursor.execute("""SELECT 
    c.competition_id,
    c.competition_name,
    cat.category_name
FROM 
    competitions c
JOIN 
    categories cat ON c.category_id = cat.category_id
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))




+----------------------+----------------------------------------------------+----------------------+
| competition_id       | competition_name                                   | category_name        |
+======================+====================================================+======================+
| sr:competition:12229 | IPTL                                               | IPTL                 |
+----------------------+----------------------------------------------------+----------------------+
| sr:competition:14814 | Juniors AO, Melbourne, Australia Men Singles       | Juniors              |
+----------------------+----------------------------------------------------+----------------------+
| sr:competition:14816 | Juniors AO, Melbourne, Australia Men Doubles       | Juniors              |
+----------------------+----------------------------------------------------+----------------------+
| sr:competition:14818 | Juniors AO, Melbourne, Australia Women Singles     | Juniors      

In [81]:
#2.Count the number of competitions in each category
mycursor.execute("""SELECT 
    c.category_id,
    cat.category_name,
    COUNT(*) AS competition_count
FROM 
    competitions c
JOIN 
    categories cat ON c.category_id = cat.category_id
GROUP BY 
     cat.category_name
ORDER BY 
    competition_count DESC
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))



+------------------+----------------------+---------------------+
| category_id      | category_name        |   competition_count |
+==================+======================+=====================+
| sr:category:785  | ITF Men              |                2198 |
+------------------+----------------------+---------------------+
| sr:category:213  | ITF Women            |                2032 |
+------------------+----------------------+---------------------+
| sr:category:72   | Challenger           |                 900 |
+------------------+----------------------+---------------------+
| sr:category:6    | WTA                  |                 253 |
+------------------+----------------------+---------------------+
| sr:category:3    | ATP                  |                 223 |
+------------------+----------------------+---------------------+
| sr:category:871  | WTA 125K             |                 172 |
+------------------+----------------------+---------------------+
| sr:categ

In [40]:
#3.Find all competitions of type 'doubles'
mycursor.execute("""
    SELECT 
        competition_id,
        competition_name,
        type
    FROM 
        competitions 
    WHERE 
        type = 'doubles'
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------------+----------------------------------------------------+---------+
| competition_id       | competition_name                                   | type    |
+======================+====================================================+=========+
| sr:competition:10027 | ITF Men Stara Zagora, Bulgaria Men Doubles         | doubles |
+----------------------+----------------------------------------------------+---------+
| sr:competition:10033 | ITF Men Sibiu, Romania Men Doubles                 | doubles |
+----------------------+----------------------------------------------------+---------+
| sr:competition:10039 | ITF Men Busto Arsizio, Italy Men Doubles           | doubles |
+----------------------+----------------------------------------------------+---------+
| sr:competition:10045 | ITF Men Sabac, Serbia Men Doubles                  | doubles |
+----------------------+----------------------------------------------------+---------+
| sr:competition:10057 | ITF Men

In [42]:
#4.Get competitions that belong to a specific category (e.g., ITF Men)
mycursor.execute("""
    SELECT 
        c.competition_id,
        c.competition_name,
        cat.category_name
    FROM 
        competitions c
    JOIN 
        categories cat ON c.category_id = cat.category_id
    WHERE 
        cat.category_name = 'ITF Men'
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------------+----------------------------------------------------+-----------------+
| competition_id       | competition_name                                   | category_name   |
+======================+====================================================+=================+
| sr:competition:10025 | ITF Men Stara Zagora, Bulgaria Men Singles         | ITF Men         |
+----------------------+----------------------------------------------------+-----------------+
| sr:competition:10027 | ITF Men Stara Zagora, Bulgaria Men Doubles         | ITF Men         |
+----------------------+----------------------------------------------------+-----------------+
| sr:competition:10031 | ITF Men Sibiu, Romania Men Singles                 | ITF Men         |
+----------------------+----------------------------------------------------+-----------------+
| sr:competition:10033 | ITF Men Sibiu, Romania Men Doubles                 | ITF Men         |
+----------------------+----------------

In [44]:
#5.Identify parent competitions and their sub-competitions
mycursor.execute("""
    SELECT 
        p.competition_id AS parent_id,
        p.competition_name AS parent_name,
        c.competition_id AS sub_id,
        c.competition_name AS sub_name,
        c.type,
        c.gender,
        c.category_id
    FROM competitions AS p
    JOIN competitions AS c 
        ON p.competition_id = c.parent_id
    ORDER BY p.competition_name, c.competition_name;
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------------+-----------------------------+----------------------+-----------------------------+---------+----------+------------------+
| parent_id            | parent_name                 | sub_id               | sub_name                    | type    | gender   | category_id      |
+======================+=============================+======================+=============================+=========+==========+==================+
| sr:competition:16976 | ITF Romania F9, Men Singles | sr:competition:16978 | ITF Romania F9, Men Doubles | doubles | men      | sr:category:785  |
+----------------------+-----------------------------+----------------------+-----------------------------+---------+----------+------------------+
| sr:competition:45363 | UTR Boca Raton W01          | sr:competition:45971 | UTR Boca Raton W03          | singles | women    | sr:category:2517 |
+----------------------+-----------------------------+----------------------+-----------------------------+-----

In [46]:
#6.Analyze the distribution of competition types by category
mycursor.execute("""
    SELECT 
        cat.category_name,
        c.type,
        COUNT(*) AS competition_count
    FROM 
        categories cat
    JOIN 
        competitions c ON c.category_id = cat.category_id
    GROUP BY 
        c.type, cat.category_name
    ORDER BY 
        cat.category_name, competition_count DESC
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))


+----------------------+---------------+---------------------+
| category_name        | type          |   competition_count |
+======================+===============+=====================+
| ATP                  | singles       |                 108 |
+----------------------+---------------+---------------------+
| ATP                  | doubles       |                 107 |
+----------------------+---------------+---------------------+
| ATP                  | mixed_doubles |                   6 |
+----------------------+---------------+---------------------+
| ATP                  | mixed         |                   2 |
+----------------------+---------------+---------------------+
| Billie Jean King Cup | mixed         |                   1 |
+----------------------+---------------+---------------------+
| Challenger           | singles       |                 451 |
+----------------------+---------------+---------------------+
| Challenger           | doubles       |               

In [48]:
#7.List all competitions with no parent (top-level competitions)
mycursor.execute("""
    SELECT 
        competition_id,
        competition_name,
        type,
        gender,
        category_id
    FROM 
        competitions
    WHERE 
        parent_id is NULL
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------------+---------------------------------------------+---------+----------+------------------+
| competition_id       | competition_name                            | type    | gender   | category_id      |
+======================+=============================================+=========+==========+==================+
| sr:competition:1207  | Championship International Series           | singles | women    | sr:category:6    |
+----------------------+---------------------------------------------+---------+----------+------------------+
| sr:competition:12229 | IPTL                                        | singles | men      | sr:category:1012 |
+----------------------+---------------------------------------------+---------+----------+------------------+
| sr:competition:15393 | ITF Men San Jose, Costa Rica Men Singles    | singles | men      | sr:category:785  |
+----------------------+---------------------------------------------+---------+----------+------------------+
|

In [71]:
mycursor.execute("select * from categories")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------+----------------------+
| category_id      | category_name        |
+==================+======================+
| sr:category:1012 | IPTL                 |
+------------------+----------------------+
| sr:category:1474 | Juniors              |
+------------------+----------------------+
| sr:category:1475 | Legends              |
+------------------+----------------------+
| sr:category:1476 | Wheelchairs          |
+------------------+----------------------+
| sr:category:181  | Hopman Cup           |
+------------------+----------------------+
| sr:category:213  | ITF Women            |
+------------------+----------------------+
| sr:category:2400 | Wheelchairs Juniors  |
+------------------+----------------------+
| sr:category:2414 | United Cup           |
+------------------+----------------------+
| sr:category:2516 | UTR Men              |
+------------------+----------------------+
| sr:category:2517 | UTR Women            |
+------------------+------------

In [67]:
mycursor.execute("select * from competitions limit 10")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------------+--------------------------------------------+----------------------+---------+----------+-----------------+
| competition_id       | competition_name                           | parent_id            | type    | gender   | category_id     |
+======================+============================================+======================+=========+==========+=================+
| sr:competition:10025 | ITF Men Stara Zagora, Bulgaria Men Singles | sr:competition:10023 | singles | men      | sr:category:785 |
+----------------------+--------------------------------------------+----------------------+---------+----------+-----------------+
| sr:competition:10027 | ITF Men Stara Zagora, Bulgaria Men Doubles | sr:competition:10023 | doubles | men      | sr:category:785 |
+----------------------+--------------------------------------------+----------------------+---------+----------+-----------------+
| sr:competition:10031 | ITF Men Sibiu, Romania Men Singles         | sr:com